In [1]:
import sys
from datetime import date, datetime, timedelta
import time
import socket
import select
import logging
from typing import Sequence
import os
import pandas as pd
import csv

In [2]:
log = logging.getLogger(__name__)
log.addHandler(logging.NullHandler())
path = 'Equities Intraday/'

In [3]:
def _is_iqfeed_running(
  iqfeed_host: str="127.0.0.1",
  iqfeed_ports: Sequence=(9300, 5009, 9100, 9200, 9400)) -> bool:
  """
  Return true if you can connect to iqfeed_sockets
  Make sure the admin_port is the first port in iqfeed_ports.The admin
  port starts listening first. You need not list all ports if you are sure
  you won't be talking to some of them.
  
  :param iqfeed_host: The host on which IQFeed is running
  :param iqfeed_ports:
  :return: True if IQFeed is running
  
  """
  try:
    for port in iqfeed_ports:
      s = socket.create_connection((iqfeed_host, port), 5)
      rl = select.select([s], [], [], 0.5)
      while rl[0]:
        # Read startup messages otherwise you get a zombie socket
        s.recv(16384)
        rl = select.select([s], [], [], 0.5)
      s.shutdown(socket.SHUT_RDWR)
      s.close()
    return True
  except ConnectionRefusedError:
    return False

In [4]:
def launch(timeout: int=20):
  if not _is_iqfeed_running():
    ShellExecute = __import__('win32api').ShellExecute
    SW_SHOWNORMAL = __import__('win32con').SW_SHOWNORMAL
    ShellExecute(0, "open", "IQTimeAndSales.exe", None, "", SW_SHOWNORMAL)
    start_time = time.time()
    while not _is_iqfeed_running():
      time.sleep(1)
      if time.time() - start_time > timeout:
        raise RuntimeError("Launching IQFeed timed out.")
  else:
    log.warning(
        "Not launching IQFeed because it is already running.")

In [5]:
def read_historical_data_socket(sock, recv_buffer=4096):
  """
  Read the information from the socket, in a buffered
  fashion, receiving only 4096 bytes at a time
    
  Parameters:
  sock - the socket object
  recv_buffer - Amount in bytes to receive per read
  """
  
  buffer = ""
  data = ""
  while True:
    data = sock.recv(recv_buffer).decode()
    buffer += data
    
    # Check if the end message string arrives
    if "!ENDMSG!" in buffer:
      break
    
  # Remove the end message string
  buffer = buffer[:-12]
  return buffer

In [6]:
launch()

In [7]:
# Define server host and port

host = "127.0.0.1"  # Localhost
port = 9100  # Historical data socket port 

In [8]:
# Open and read list of tickers from NASDAQ FTP site

url = "ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt"
df = pd.read_csv(url, sep="|")
file_update_str = df['Nasdaq Traded'].iloc[-1].split(' ')[-1]
file_update_time = datetime.strptime(file_update_str, '%m%d%Y%H:%M')

symbol_list = list(df['Symbol'][(df['ETF']=="N") & (df['Test Issue']=='N')])
symbol_list = [sym.replace('$','-') for sym in symbol_list]
symbol_list = [sym.replace('.W','.WS') for sym in symbol_list]
symbol_list = [sym.replace('NSPR.B','NSPR.WS.B') for sym in symbol_list]
symbol_list = [sym.replace('TDW.A','TDW.WS.A') for sym in symbol_list]
symbol_list = [sym.replace('TDW.B','TDW.WS.B') for sym in symbol_list]
symbol_list = [sym.replace('VST.A','VST.WS.A') for sym in symbol_list]

print(str(len(symbol_list)) + " symbols in list. Last updated: " + str(file_update_time))

6740 symbols in list. Last updated: 2020-08-20 08:03:00


In [9]:
# Get list of symbols in filepath currently containing data

filelist = [f for f in os.listdir(path)]
existing_symbols = []
for file in filelist:
    existing_symbols.append(file.split("_")[0])

# Get lists of new symbols, symbols to update, and delisted symbols
new_symbols = [s for s in symbol_list if s not in existing_symbols]
symbols_to_update = [s for s in symbol_list if s in existing_symbols]
delisted = [s for s in existing_symbols if s not in symbol_list]

print(str(len(new_symbols)) + ' new symbols found: ' + ', '.join(new_symbols))
print(str(len(symbols_to_update)) + ' existing symbols to update.')
print(str(len(delisted)) + ' delisted tickers: ' + ', '.join(delisted))

15 new symbols found: AEP-C, AOUTV, BFT.U, EVLMC, FIIIU, GSAH, GSAH.WS, HRMY, INBX, KTRA, MAXNV, PSA-M, SVA, SWBIV, UZD
6725 existing symbols to update.
9 delisted tickers: CHAP, DBCP, DMPI, FSB, FSCT, GBLIZ, PPHI, RYCE, VAL


In [10]:
# Get today's and yesterday's dates and the current time

today = date.today()
yesterday = today - timedelta(days = 1)
now = datetime.now()

In [11]:
# Download each new symbol to disk

sym_count = 0
err_count = 0
skipped_symbols = []

for sym in new_symbols:
    print(f'Downloading symbol: {sym}...', end='')
    
    # Construct the message needed by IQFeed to retrieve initial data
    message = "HIT," + sym + ",60,20200101 040000," + yesterday.strftime("%Y%m%d") + " 200000,,,,1\r\n"

    # Open a streaming socket to the IQFeed server locally
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((host, port))
    
    # Send the historical data request message and buffer the data
    sock.sendall(message.encode())
    data = read_historical_data_socket(sock)
    sock.close
    
    # Remove all the endlines and line-ending comma delimiter from each record
    data = "".join(data.split("\r"))
    data = data.replace(",\n","\n")[:-1]
    
    # Skip the symbol without saving if no data is returned
    if "!NO_DATA!" in data:
        print('NO DATA for symbol')
        err_count += 1
        skipped_symbols.append(sym)
        continue
    
    #Write the data stream to disk
    f = open("Equities Intraday/%s_1.csv" % sym, "w")
    f.write(data)
    f.close
    print("Complete")
    sym_count += 1
    
print(f'Data download complete...{sym_count} symbols added, {err_count} symbols with no data as of {yesterday}.')

Data download complete...7 symbols added, 8 symbols with no data as of 2020-08-19.


In [12]:
# Append new data to previously downloaded symbols

sym_count = 0
err_count = 0
skipped_symbols = []

for sym in symbols_to_update:
    print(f'Updating symbol: {sym}...', end='')
    
    df = pd.read_csv(f'{path}{sym}_1.csv', header=None)
    last_data = datetime.strptime(df.iloc[-1][0], '%Y-%m-%d %H:%M:%S')

    # Construct the message needed by IQFeed to retrieve data update
    message = "HIT," + sym + ",60," + last_data.strftime("%Y%m%d %H%M%S") + "," + yesterday.strftime("%Y%m%d") + " 200000,,,,1\r\n"

    # Open a streaming socket to the IQFeed server locally
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((host, port))
    
    # Send the historical data request message and buffer the data
    sock.sendall(message.encode())
    data = read_historical_data_socket(sock)
    sock.close

    # Remove all the endlines and line-ending comma delimiter from each record
    data = "".join(data.split("\r"))
    data = data.replace(",\n","\n")[:-1]
    
    
    # Skip the symbol without saving if no data is returned
    if "!NO_DATA!" in data:
        print('NO NEW DATA AVAILABLE')
        err_count += 1
        skipped_symbols.append(sym)
        continue

    #Append today's data to the existing file
    f = open("Equities Intraday/%s_1.csv" % sym, "a")
    f.write("\r" + data)
    f.close
    print("Complete")
    sym_count += 1
    
print(f'Data update complete...{sym_count} symbols updated, {err_count} symbols with no new data.')

Updating symbol: A...Complete
Updating symbol: AA...Complete
Updating symbol: AACG...Complete
Updating symbol: AACQU...Complete
Updating symbol: AAL...Complete
Updating symbol: AAMC...Complete
Updating symbol: AAME...Complete
Updating symbol: AAN...Complete
Updating symbol: AAOI...Complete
Updating symbol: AAON...Complete
Updating symbol: AAP...Complete
Updating symbol: AAPL...Complete
Updating symbol: AAT...Complete
Updating symbol: AAU...Complete
Updating symbol: AAWW...Complete
Updating symbol: AAXN...Complete
Updating symbol: AB...Complete
Updating symbol: ABB...Complete
Updating symbol: ABBV...Complete
Updating symbol: ABC...Complete
Updating symbol: ABCB...Complete
Updating symbol: ABEO...Complete
Updating symbol: ABEV...Complete
Updating symbol: ABG...Complete
Updating symbol: ABIO...Complete
Updating symbol: ABM...Complete
Updating symbol: ABMD...Complete
Updating symbol: ABR...Complete
Updating symbol: ABR-A...Complete
Updating symbol: ABR-B...Complete
Updating symbol: ABR-C..

Updating symbol: ALEX...Complete
Updating symbol: ALG...Complete
Updating symbol: ALGN...Complete
Updating symbol: ALGT...Complete
Updating symbol: ALIM...Complete
Updating symbol: ALIN-A...Complete
Updating symbol: ALIN-B...Complete
Updating symbol: ALIN-E...Complete
Updating symbol: ALJJ...Complete
Updating symbol: ALK...Complete
Updating symbol: ALKS...Complete
Updating symbol: ALL...Complete
Updating symbol: ALL-B...Complete
Updating symbol: ALL-G...Complete
Updating symbol: ALL-H...Complete
Updating symbol: ALL-I...Complete
Updating symbol: ALLE...Complete
Updating symbol: ALLK...Complete
Updating symbol: ALLO...Complete
Updating symbol: ALLT...Complete
Updating symbol: ALLY...Complete
Updating symbol: ALLY-A...Complete
Updating symbol: ALNA...Complete
Updating symbol: ALNY...Complete
Updating symbol: ALOT...Complete
Updating symbol: ALP-Q...Complete
Updating symbol: ALPN...Complete
Updating symbol: ALRM...Complete
Updating symbol: ALRN...Complete
Updating symbol: ALRS...Complete


Updating symbol: ASG...Complete
Updating symbol: ASGI...Complete
Updating symbol: ASGN...Complete
Updating symbol: ASH...Complete
Updating symbol: ASIX...Complete
Updating symbol: ASLN...Complete
Updating symbol: ASM...Complete
Updating symbol: ASMB...Complete
Updating symbol: ASML...Complete
Updating symbol: ASND...Complete
Updating symbol: ASPN...Complete
Updating symbol: ASPS...Complete
Updating symbol: ASPU...Complete
Updating symbol: ASR...Complete
Updating symbol: ASRT...Complete
Updating symbol: ASRV...Complete
Updating symbol: ASRVP...Complete
Updating symbol: ASTC...Complete
Updating symbol: ASTE...Complete
Updating symbol: ASUR...Complete
Updating symbol: ASX...Complete
Updating symbol: ASYS...Complete
Updating symbol: AT...Complete
Updating symbol: ATAX...Complete
Updating symbol: ATCO...Complete
Updating symbol: ATCO-D...Complete
Updating symbol: ATCO-E...Complete
Updating symbol: ATCO-G...Complete
Updating symbol: ATCO-H...Complete
Updating symbol: ATCO-I...Complete
Updati

Updating symbol: BERY...Complete
Updating symbol: BEST...Complete
Updating symbol: BF.A...Complete
Updating symbol: BF.B...Complete
Updating symbol: BFAM...Complete
Updating symbol: BFC...Complete
Updating symbol: BFIN...Complete
Updating symbol: BFK...Complete
Updating symbol: BFO...Complete
Updating symbol: BFRA...Complete
Updating symbol: BFS...Complete
Updating symbol: BFS-D...Complete
Updating symbol: BFS-E...Complete
Updating symbol: BFST...Complete
Updating symbol: BFY...Complete
Updating symbol: BFYT...Complete
Updating symbol: BFZ...Complete
Updating symbol: BG...Complete
Updating symbol: BGB...Complete
Updating symbol: BGCP...Complete
Updating symbol: BGFV...Complete
Updating symbol: BGH...Complete
Updating symbol: BGI...Complete
Updating symbol: BGIO...Complete
Updating symbol: BGNE...Complete
Updating symbol: BGR...Complete
Updating symbol: BGS...Complete
Updating symbol: BGSF...Complete
Updating symbol: BGT...Complete
Updating symbol: BGX...Complete
Updating symbol: BGY...

Updating symbol: BVN...Complete
Updating symbol: BVXV...Complete
Updating symbol: BW...Complete
Updating symbol: BWA...Complete
Updating symbol: BWAY...Complete
Updating symbol: BWB...Complete
Updating symbol: BWEN...Complete
Updating symbol: BWFG...Complete
Updating symbol: BWG...Complete
Updating symbol: BWL.A...Complete
Updating symbol: BWMX...Complete
Updating symbol: BWXT...Complete
Updating symbol: BX...Complete
Updating symbol: BXC...Complete
Updating symbol: BXG...Complete
Updating symbol: BXMT...Complete
Updating symbol: BXMX...Complete
Updating symbol: BXP...Complete
Updating symbol: BXP-B...Complete
Updating symbol: BXRX...Complete
Updating symbol: BXS...Complete
Updating symbol: BXS-A...Complete
Updating symbol: BY...Complete
Updating symbol: BYD...Complete
Updating symbol: BYFC...Complete
Updating symbol: BYM...Complete
Updating symbol: BYND...Complete
Updating symbol: BYSI...Complete
Updating symbol: BZH...Complete
Updating symbol: BZM...Complete
Updating symbol: BZUN...C

...

Updating symbol: SPAQ.WS...Complete
Updating symbol: SPB...Complete
Updating symbol: SPCB...Complete
Updating symbol: SPCE...Complete
Updating symbol: SPE...Complete
Updating symbol: SPE-B...NO NEW DATA AVAILABLE
Updating symbol: SPFI...Complete
Updating symbol: SPG...Complete
Updating symbol: SPG-J...Complete
Updating symbol: SPGI...Complete
Updating symbol: SPH...Complete
Updating symbol: SPI...Complete
Updating symbol: SPKE...Complete
Updating symbol: SPKEP...Complete
Updating symbol: SPLK...Complete
Updating symbol: SPLP...Complete
Updating symbol: SPLP-A...Complete
Updating symbol: SPN...Complete
Updating symbol: SPNE...Complete
Updating symbol: SPNS...Complete
Updating symbol: SPOK...Complete
Updating symbol: SPOT...Complete
Updating symbol: SPPI...Complete
Updating symbol: SPR...Complete
Updating symbol: SPRO...Complete
Updating symbol: SPRT...Complete
Updating symbol: SPSC...Complete
Updating symbol: SPT...Complete
Updating symbol: SPTN...Complete
Updating symbol: SPWH...Comple

Updating symbol: TDAC...Complete
Updating symbol: TDACU...NO NEW DATA AVAILABLE
Updating symbol: TDACW...Complete
Updating symbol: TDC...Complete
Updating symbol: TDE...Complete
Updating symbol: TDF...Complete
Updating symbol: TDG...Complete
Updating symbol: TDI...Complete
Updating symbol: TDJ...Complete
Updating symbol: TDOC...Complete
Updating symbol: TDS...Complete
Updating symbol: TDW...Complete
Updating symbol: TDW.WS.A...Complete
Updating symbol: TDW.WS.B...Complete
Updating symbol: TDW.WS...NO NEW DATA AVAILABLE
Updating symbol: TDY...Complete
Updating symbol: TEAF...Complete
Updating symbol: TEAM...Complete
Updating symbol: TECH...Complete
Updating symbol: TECK...Complete
Updating symbol: TECTP...NO NEW DATA AVAILABLE
Updating symbol: TEDU...Complete
Updating symbol: TEF...Complete
Updating symbol: TEI...Complete
Updating symbol: TEL...Complete
Updating symbol: TELA...Complete
Updating symbol: TELL...Complete
Updating symbol: TEN...Complete
Updating symbol: TENB...Complete
Upda

Updating symbol: TWO-A...Complete
Updating symbol: TWO-B...Complete
Updating symbol: TWO-C...Complete
Updating symbol: TWO-D...Complete
Updating symbol: TWO-E...Complete
Updating symbol: TWOU...Complete
Updating symbol: TWST...Complete
Updating symbol: TWTR...Complete
Updating symbol: TX...Complete
Updating symbol: TXG...Complete
Updating symbol: TXMD...Complete
Updating symbol: TXN...Complete
Updating symbol: TXRH...Complete
Updating symbol: TXT...Complete
Updating symbol: TY...Complete
Updating symbol: TY-...Complete
Updating symbol: TYG...Complete
Updating symbol: TYHT...Complete
Updating symbol: TYL...Complete
Updating symbol: TYME...Complete
Updating symbol: TZAC...Complete
Updating symbol: TZACU...NO NEW DATA AVAILABLE
Updating symbol: TZACW...Complete
Updating symbol: TZOO...Complete
Updating symbol: UA...Complete
Updating symbol: UAA...Complete
Updating symbol: UAG...Complete
Updating symbol: UAL...Complete
Updating symbol: UAMY...Complete
Updating symbol: UAN...Complete
Updati

Updating symbol: VOYA...Complete
Updating symbol: VOYA-B...Complete
Updating symbol: VPG...Complete
Updating symbol: VPV...Complete
Updating symbol: VQT...Complete
Updating symbol: VRA...Complete
Updating symbol: VRAY...Complete
Updating symbol: VRCA...Complete
Updating symbol: VREX...Complete
Updating symbol: VRM...Complete
Updating symbol: VRME...Complete
Updating symbol: VRMEW...Complete
Updating symbol: VRNA...Complete
Updating symbol: VRNS...Complete
Updating symbol: VRNT...Complete
Updating symbol: VRRM...Complete
Updating symbol: VRS...Complete
Updating symbol: VRSK...Complete
Updating symbol: VRSN...Complete
Updating symbol: VRT...Complete
Updating symbol: VRT.WS...Complete
Updating symbol: VRTS...Complete
Updating symbol: VRTU...Complete
Updating symbol: VRTV...Complete
Updating symbol: VRTX...Complete
Updating symbol: VSAT...Complete
Updating symbol: VSEC...Complete
Updating symbol: VSH...Complete
Updating symbol: VSLR...Complete
Updating symbol: VST...Complete
Updating symbo

Updating symbol: XPO...Complete
Updating symbol: XRAY...Complete
Updating symbol: XRX...Complete
Updating symbol: XSPA...Complete
Updating symbol: XTLB...Complete
Updating symbol: XTNT...Complete
Updating symbol: XVZ...Complete
Updating symbol: XXII...Complete
Updating symbol: XYF...Complete
Updating symbol: XYL...Complete
Updating symbol: Y...Complete
Updating symbol: YAC.U...Complete
Updating symbol: YCBD...Complete
Updating symbol: YCBD-A...NO NEW DATA AVAILABLE
Updating symbol: YELP...Complete
Updating symbol: YETI...Complete
Updating symbol: YEXT...Complete
Updating symbol: YGRN...Complete
Updating symbol: YGYI...Complete
Updating symbol: YGYIP...Complete
Updating symbol: YI...Complete
Updating symbol: YIN...Complete
Updating symbol: YJ...Complete
Updating symbol: YMAB...Complete
Updating symbol: YNDX...Complete
Updating symbol: YORW...Complete
Updating symbol: YPF...Complete
Updating symbol: YRCW...Complete
Updating symbol: YRD...Complete
Updating symbol: YTEN...Complete
Updating

In [13]:
# load and format dataframe
df = pd.read_csv(f'{path}AAPL_1.csv', header=None, index_col=0)
df.columns = ['High','Low','Open','Close','Volume','Inc Vol']
df.index.rename('Timestamp', inplace=True)
df.index = pd.to_datetime(df.index)

In [14]:
# Get daily OHLC and Volume
ohlc = df.between_time('09:30','16:00', include_start=False, include_end=True).resample('D').agg(
                       {'Open': 'first',
                        'High': 'max',
                        'Low': 'min',
                        'Close': 'last'}).dropna()

volume = df['Volume'].resample('D').last().dropna()

# Get pre-market OHLC and Cumulative Volume
ohlcv_premkt = df.between_time('04:00','09:30', include_start=False, 
                               include_end=True).resample('D').agg({'Open': 'first',
                                                                    'High': 'max',
                                                                    'Low': 'min',
                                                                    'Close': 'last',
                                                                    'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for 1st minute of RTH
ohlcv_1 = df.between_time('09:30','09:31', include_start=False, 
                          include_end=True).resample('D').agg({'Open': 'first',
                                                               'High': 'max',
                                                               'Low': 'min',
                                                               'Close': 'last',
                                                               'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for 1st 5 minutes of RTH
ohlcv_5 = df.between_time('09:30','09:35', include_start=False, 
                          include_end=True).resample('D').agg({'Open': 'first',
                                                               'High': 'max',
                                                               'Low': 'min',
                                                               'Close': 'last',
                                                               'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for 1st 15 minutes of RTH
ohlcv_15 = df.between_time('09:30','09:45', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for 1st 30 minutes of RTH
ohlcv_30 = df.between_time('09:30','10:00', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for 1st 60 minutes of RTH
ohlcv_60 = df.between_time('09:30','10:30', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume before Noon ET RTH
ohlcv_am = df.between_time('09:30','12:00', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume after Noon ET RTH
ohlcv_pm = df.between_time('12:00','16:00', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Get OHLC and Cumulative Volume for after-hours session
ohlcv_ah = df.between_time('16:00','20:00', include_start=False, 
                           include_end=True).resample('D').agg({'Open': 'first',
                                                                'High': 'max',
                                                                'Low': 'min',
                                                                'Close': 'last',
                                                                'Volume': 'last'}).dropna()

# Rename columns
ohlcv_premkt.columns = ['PremktO', 'PremktH', 'PremktL', 'PremktC', 'PremktCumVol']
ohlcv_1.columns = ['O_1m', 'H_1m', 'L_1m', 'C_1m', 'CumVol_1m']
ohlcv_5.columns = ['O_5m', 'H_5m', 'L_5m', 'C_5m', 'CumVol_5m']
ohlcv_15.columns = ['O_15m', 'H_15m', 'L_15m', 'C_15m', 'CumVol_15m']
ohlcv_30.columns = ['O_30m', 'H_30m', 'L_30m', 'C_30m', 'CumVol_30m']
ohlcv_60.columns = ['O_60m', 'H_60m', 'L_60m', 'C_60m', 'CumVol_60m']
ohlcv_am.columns = ['O_am', 'H_am', 'L_am', 'C_am', 'CumVol_am']
ohlcv_pm.columns = ['O_pm', 'H_pm', 'L_pm', 'C_pm', 'CumVol_pm']
ohlcv_ah.columns = ['O_ah', 'H_ah', 'L_ah', 'C_ah', 'CumVol_ah']

# Calculate incremental volume for all timeframes
ohlcv_premkt['PremktVol'] = ohlcv_premkt['PremktCumVol']
ohlcv_1['Vol_1m'] = ohlcv_1['CumVol_1m'] - ohlcv_premkt['PremktVol']
ohlcv_5['Vol_5m'] = ohlcv_5['CumVol_5m'] - ohlcv_premkt['PremktVol']
ohlcv_15['Vol_15m'] = ohlcv_15['CumVol_15m'] - ohlcv_premkt['PremktVol']
ohlcv_30['Vol_30m'] = ohlcv_30['CumVol_30m'] - ohlcv_premkt['PremktVol']
ohlcv_60['Vol_60m'] = ohlcv_60['CumVol_60m'] - ohlcv_premkt['PremktVol']
ohlcv_am['Vol_am'] = ohlcv_am['CumVol_am'] - ohlcv_premkt['PremktVol']
ohlcv_pm['Vol_pm'] = ohlcv_pm['CumVol_pm'] - ohlcv_am['CumVol_am']
ohlcv_ah['Vol_ah'] = ohlcv_ah['CumVol_ah'] - ohlcv_pm['CumVol_pm']

In [15]:
# Get previous and next day's OHLCV
prevO = ohlc['Open'].shift(1).rename('PrevO')
prevH = ohlc['High'].shift(1).rename('PrevH')
prevL = ohlc['Low'].shift(1).rename('PrevL')
prevC = ohlc['Close'].shift(1).rename('PrevC')
prevVol = volume.shift(1).rename('PrevVol')

nextO = ohlc['Open'].shift(-1).rename('NextO')
nextH = ohlc['High'].shift(-1).rename('NextH')
nextL = ohlc['Low'].shift(-1).rename('NextL')
nextC = ohlc['Close'].shift(-1).rename('NextC')
nextVol = volume.shift(-1).rename('NextVol')

In [16]:
df['Date'] = df.index.date

# Get time of Pre-market High and Low
premkt_high_time = df[['Date','High']].between_time('04:00','09:30', include_start=False, include_end=True).groupby('Date').idxmax()
premkt_high_time.index = pd.to_datetime(premkt_high_time.index)
premkt_high_time = premkt_high_time['High'].dt.time.rename('PreMktH_Time')

premkt_low_time = df[['Date','Low']].between_time('04:00','09:30', include_start=False, include_end=True).groupby('Date').idxmin()
premkt_low_time.index = pd.to_datetime(premkt_low_time.index)
premkt_low_time = premkt_low_time['Low'].dt.time.rename('PreMktL_Time')


# Get time of Reg. Trading Hours High and Low
high_time = df[['Date','High']].between_time('09:30','16:00', include_start=False, include_end=True).groupby('Date').idxmax()
high_time.index = pd.to_datetime(high_time.index)
high_time = high_time['High'].dt.time.rename('High_Time')

low_time = df[['Date','Low']].between_time('09:30','16:00', include_start=False, include_end=True).groupby('Date').idxmin()
low_time.index = pd.to_datetime(low_time.index)
low_time = low_time['Low'].dt.time.rename('Low_Time')


# Get time of after-hours High and Low
ah_high_time = df[['Date','High']].between_time('16:00','20:00', include_start=False, include_end=True).groupby('Date').idxmax()
ah_high_time.index = pd.to_datetime(ah_high_time.index)
ah_high_time = ah_high_time['High'].dt.time.rename('H_ah_Time')

ah_low_time = df[['Date','Low']].between_time('16:00','20:00', include_start=False, include_end=True).groupby('Date').idxmin()
ah_low_time.index = pd.to_datetime(ah_low_time.index)
ah_low_time = ah_low_time['Low'].dt.time.rename('L_ah_Time')

In [18]:
def extrema(df_original, start_time_str, end_time_str):
    # clean the data once at start
    df = df_original.between_time(
        start_time_str, end_time_str, include_start=False, include_end=True)
    
    # your methods for max/min
    daily = df.resample('D').agg({'High':'max', 'Low':'min'}).dropna()
    daily['High_Time'] = df.groupby(df.index.date).High.idxmax().dt.time
    daily['Low_Time'] = df.groupby(df.index.date).Low.idxmin().dt.time
    
    hal, lah, hal_time, lah_time = [], [], [], []
    for (date, rows), (_, high, low, htime, ltime) in zip(
            df.groupby(df.index.date), daily.itertuples()):
        if htime > ltime:
            # high_after_low == high, no need to search again
            hal_time.append(htime)
            hal.append(high)
            # get low_after_high
            if htime == rows.index[-1].time():
                lah_time.append(htime)
                lah.append(high)
            else:
                t = rows.loc[
                    rows.index > datetime.combine(date, htime), 'Low'].idxmin()
                lah_time.append(t.time())
                lah.append(rows.loc[t, 'Low'])
        else:
            # low_after_high == low
            lah.append(low)
            lah_time.append(ltime)
            # get high_after_low
            if ltime == rows.index[-1].time():
                hal_time.append(ltime)
                hal.append(low)
            else:
                t = rows.loc[
                    rows.index > datetime.combine(date, ltime), 'High'].idxmax()
                hal_time.append(t.time())
                hal.append(rows.loc[t, 'High'])
    daily = pd.concat([daily,
        pd.DataFrame(
            {'High_after_Low': hal, 'Low_after_High': lah,
                'High_after_Low_Time': hal_time, 'Low_after_High_Time': lah_time},
            index=daily.index)
        ], axis=1)
    
    return daily

result = extrema(df, '09:30', '16:00')
print(result)

                High       Low High_Time  Low_Time  High_after_Low  \
Timestamp                                                            
2020-01-02  300.6000  295.1900  16:00:00  09:33:00        300.6000   
2020-01-03  300.5800  296.5000  12:52:00  09:31:00        300.5800   
2020-01-06  299.9600  292.7501  14:15:00  09:31:00        299.9600   
2020-01-07  300.9000  297.4800  09:35:00  10:29:00        300.4100   
2020-01-08  304.4399  297.1560  15:42:00  09:31:00        304.4399   
...              ...       ...       ...       ...             ...   
2020-08-13  464.1700  455.7100  13:01:00  10:19:00        464.1700   
2020-08-14  460.0000  452.1800  15:56:00  11:05:00        460.0000   
2020-08-17  464.3600  455.8501  09:31:00  11:47:00        460.9100   
2020-08-18  464.0000  456.0300  14:24:00  10:31:00        464.0000   
2020-08-19  468.6500  462.4400  11:05:00  15:56:00        463.4900   

            Low_after_High High_after_Low_Time Low_after_High_Time  
Timestamp           